# IMPORTS

In [ ]:
import sys, importlib
importlib.reload(sys.modules['initialize_functions'])
importlib.reload(sys.modules['request_functions'])
#importlib.reload(sys.modules['firebase_functions'])

In [1]:
import json
from random import randint
from datetime import datetime, timedelta

from initialize_functions import *
import initialize_functions as init_fun

from request_functions import *
import request_functions as req_fun

from firebase_functions import *
import firebase_functions as fb_fun

# LISTA TODAS AS VAGAS

In [2]:
def getSpotsName():
    parking_spots = json.loads(listParkingSpots())
    parking_spot_list = []
    for spot in parking_spots:
        parking_spot_list.append(spot["id"])
    return parking_spot_list

# CONTROLA VAGAS

In [3]:
def controlParkingRental(plate_number, spot_id):
    rental = getCurrentRental(plate_number)
    if len(rental) < 3:
        return openNewRental(plate_number, spot_id) 
        
def controlParkingSpotStatus(spot_id, plate_number):
    #resgata a placa identificada na última revisão (None caso estivesse vazia na última verificação)     
    try:
        last_plate = json.loads(checkSpot(spot_id))["plate"]["plateNumber"]
    except:
        last_plate = None
        
    #caso a placa anterios seja igual a placa atual, não faz nada    
    if last_plate == plate_number:
        print("NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA")
        print("ATUAL: {} - ANTERIOR: {}".format(plate_number, last_plate))
        return
    
    #caso não seja identificada uma placa, altera status da vaga para disponível 
    if not plate_number:
        setParkingSpotStatusAvailable(spot_id)
        print("A VAGA ESTÁ LIVRE AGORA")
        print("ATUAL: {} - ANTERIOR: {}".format(plate_number, last_plate))
        return
        
    #caso seja identificada uma placa, altera status da vaga para ocupada, e atribui número da placa a vaga
    if plate_number:
        setParkingSpotStatusUnavailable(spot_id, plate_number)
        print("A VAGA ESTÁ OCUPADA AGORA")
        print("ATUAL: {} - ANTERIOR: {}".format(plate_number, last_plate))
        controlParkingRental(plate_number, spot_id) 
        return

In [4]:
def verifySpots(spot_id):
    #resgata imagem do firebase
    for i in [1, 2, 3]:
        spot_image = get_image("parking_spots", spot_id)
        if spot_image:
            continue
        time.sleep(1)
    
    #seguimenta a placa
    prepared_IMG = prepareIMG(spot_image)

    #aplica ocr
    ocr_res = ocr(prepared_IMG[0], prepared_IMG[1], 0.7)

    #formata resultado do ocr
    plate_number = format_ocr(ocr_res)

    #seta status da vaga
    controlParkingSpotStatus(spot_id, plate_number)

    print("{} ESTÁ NA VAGA {}".format(plate_number, spot_id))

# CONTROLA SAÍDA

In [5]:
def finishParkingRental(plate_number):
    rental = getCurrentRental(plate_number)
    if rental:
        finishParkingRental(plate_number)

In [6]:
def verifyExit():
    #resgata imagem do firebase
    exit_image = get_image("exit", 0)
    
    for i in [1, 2, 3]:
        exit_image = get_image("exit", 0)
        if exit_image:
            continue
        time.sleep(1)
    
    #seguimenta a placa
    prepared_IMG = prepareIMG(exit_image)

    #aplica ocr
    ocr_res = ocr(prepared_IMG[0], prepared_IMG[1], 0.7)

    #formata resultado do ocr
    plate_number = format_ocr(ocr_res)
    
    if plate_number: #<<<<<<<<<<<<<<
        #finaliza estadia do cliente
        closeRental(plate_number)

        print("{} ESTÁ NA SAÍDA".format(plate_number))
        return
    
    print("NENHUM VEÍCULO ESTÁ NA SAÍDA")

# MAIN

In [8]:
def main():
    parking_spot_id_list = getSpotsName()
    while True:
        for spot_id in parking_spot_id_list:
            print("==================================================")
            print("VERIFICANDO VAGA {}:".format(spot_id))
            verifySpots(spot_id)
            
            print("==================================================")
            print("VERIFICANDO SAÍDA")
            verifyExit()

In [ ]:
main()

VERIFICANDO VAGA 2:
A VAGA ESTÁ OCUPADA AGORA
ATUAL: BEE4R22 - ANTERIOR: None
BEE4R22 ESTÁ NA VAGA 2
VERIFICANDO VAGA 1:
A VAGA ESTÁ OCUPADA AGORA
ATUAL: BRA2E19 - ANTERIOR: None
BRA2E19 ESTÁ NA VAGA 1
VERIFICANDO VAGA 3:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: None - ANTERIOR: None
None ESTÁ NA VAGA 3
VERIFICANDO VAGA 2:
A VAGA ESTÁ LIVRE AGORA
ATUAL: None - ANTERIOR: BEE4R22
None ESTÁ NA VAGA 2
VERIFICANDO VAGA 1:
A VAGA ESTÁ LIVRE AGORA
ATUAL: None - ANTERIOR: BRA2E19
None ESTÁ NA VAGA 1
VERIFICANDO VAGA 3:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: None - ANTERIOR: None
None ESTÁ NA VAGA 3
VERIFICANDO VAGA 2:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: None - ANTERIOR: None
None ESTÁ NA VAGA 2
VERIFICANDO VAGA 1:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: None - ANTERIOR: None
None ESTÁ NA VAGA 1
VERIFICANDO VAGA 3:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: None - ANTERIOR: None
None ESTÁ NA VAGA 3
VERIFICANDO VAGA 2:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: None -

NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: None - ANTERIOR: None
None ESTÁ NA VAGA 1
VERIFICANDO VAGA 3:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: None - ANTERIOR: None
None ESTÁ NA VAGA 3
VERIFICANDO VAGA 2:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: None - ANTERIOR: None
None ESTÁ NA VAGA 2
VERIFICANDO VAGA 1:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: None - ANTERIOR: None
None ESTÁ NA VAGA 1
VERIFICANDO VAGA 3:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: None - ANTERIOR: None
None ESTÁ NA VAGA 3
VERIFICANDO VAGA 2:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: None - ANTERIOR: None
None ESTÁ NA VAGA 2
VERIFICANDO VAGA 1:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: None - ANTERIOR: None
None ESTÁ NA VAGA 1
VERIFICANDO VAGA 3:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: None - ANTERIOR: None
None ESTÁ NA VAGA 3
VERIFICANDO VAGA 2:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: None - ANTERIOR: None
None ESTÁ NA VAGA 2
VERIFICANDO VAGA 1:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA

NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: None - ANTERIOR: None
None ESTÁ NA VAGA 3
VERIFICANDO VAGA 2:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: BEE4R22 - ANTERIOR: BEE4R22
BEE4R22 ESTÁ NA VAGA 2
VERIFICANDO VAGA 1:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: None - ANTERIOR: None
None ESTÁ NA VAGA 1
VERIFICANDO VAGA 3:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: None - ANTERIOR: None
None ESTÁ NA VAGA 3
VERIFICANDO VAGA 2:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: BEE4R22 - ANTERIOR: BEE4R22
BEE4R22 ESTÁ NA VAGA 2
VERIFICANDO VAGA 1:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: None - ANTERIOR: None
None ESTÁ NA VAGA 1
VERIFICANDO VAGA 3:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: None - ANTERIOR: None
None ESTÁ NA VAGA 3
VERIFICANDO VAGA 2:

A VAGA ESTÁ LIVRE AGORA
ATUAL: None - ANTERIOR: BEE4R22
None ESTÁ NA VAGA 2
VERIFICANDO VAGA 1:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: None - ANTERIOR: None
None ESTÁ NA VAGA 1
VERIFICANDO VAGA 3:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO

BEE4R22 ESTÁ NA VAGA 2
VERIFICANDO VAGA 1:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: None - ANTERIOR: None
None ESTÁ NA VAGA 1
VERIFICANDO VAGA 3:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: None - ANTERIOR: None
None ESTÁ NA VAGA 3
VERIFICANDO VAGA 2:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: BEE4R22 - ANTERIOR: BEE4R22
BEE4R22 ESTÁ NA VAGA 2
VERIFICANDO VAGA 1:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: None - ANTERIOR: None
None ESTÁ NA VAGA 1
VERIFICANDO VAGA 3:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: None - ANTERIOR: None
None ESTÁ NA VAGA 3
VERIFICANDO VAGA 2:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: BEE4R22 - ANTERIOR: BEE4R22
BEE4R22 ESTÁ NA VAGA 2
VERIFICANDO VAGA 1:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: None - ANTERIOR: None
None ESTÁ NA VAGA 1
VERIFICANDO VAGA 3:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: None - ANTERIOR: None
None ESTÁ NA VAGA 3
VERIFICANDO VAGA 2:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: BEE4R22 - ANTERIOR: BEE4R22
BEE4R22 ES

NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: None - ANTERIOR: None
None ESTÁ NA VAGA 3
VERIFICANDO VAGA 2:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: BEE4R22 - ANTERIOR: BEE4R22
BEE4R22 ESTÁ NA VAGA 2
VERIFICANDO VAGA 1:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: None - ANTERIOR: None
None ESTÁ NA VAGA 1
VERIFICANDO VAGA 3:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: None - ANTERIOR: None
None ESTÁ NA VAGA 3
VERIFICANDO VAGA 2:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: BEE4R22 - ANTERIOR: BEE4R22
BEE4R22 ESTÁ NA VAGA 2
VERIFICANDO VAGA 1:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: None - ANTERIOR: None
None ESTÁ NA VAGA 1
VERIFICANDO VAGA 3:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: None - ANTERIOR: None
None ESTÁ NA VAGA 3
VERIFICANDO VAGA 2:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: BEE4R22 - ANTERIOR: BEE4R22
BEE4R22 ESTÁ NA VAGA 2
VERIFICANDO VAGA 1:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: None - ANTERIOR: None
None ESTÁ NA VAGA 1
VERIFICANDO VAGA 3:
NÃO HOUVE 

NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: BEE4R22 - ANTERIOR: BEE4R22
BEE4R22 ESTÁ NA VAGA 2
VERIFICANDO VAGA 1:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: None - ANTERIOR: None
None ESTÁ NA VAGA 1
VERIFICANDO VAGA 3:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: None - ANTERIOR: None
None ESTÁ NA VAGA 3
VERIFICANDO VAGA 2:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: BEE4R22 - ANTERIOR: BEE4R22
BEE4R22 ESTÁ NA VAGA 2
VERIFICANDO VAGA 1:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: None - ANTERIOR: None
None ESTÁ NA VAGA 1
VERIFICANDO VAGA 3:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: None - ANTERIOR: None
None ESTÁ NA VAGA 3
VERIFICANDO VAGA 2:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: BEE4R22 - ANTERIOR: BEE4R22
BEE4R22 ESTÁ NA VAGA 2
VERIFICANDO VAGA 1:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: None - ANTERIOR: None
None ESTÁ NA VAGA 1
VERIFICANDO VAGA 3:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: None - ANTERIOR: None
None ESTÁ NA VAGA 3
VERIFICANDO VAGA 2:

A VAGA ES

NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: None - ANTERIOR: None
None ESTÁ NA VAGA 1
VERIFICANDO VAGA 3:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: None - ANTERIOR: None
None ESTÁ NA VAGA 3
VERIFICANDO VAGA 2:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: BEE4R22 - ANTERIOR: BEE4R22
BEE4R22 ESTÁ NA VAGA 2
VERIFICANDO VAGA 1:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: None - ANTERIOR: None
None ESTÁ NA VAGA 1
VERIFICANDO VAGA 3:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: None - ANTERIOR: None
None ESTÁ NA VAGA 3
VERIFICANDO VAGA 2:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: BEE4R22 - ANTERIOR: BEE4R22
BEE4R22 ESTÁ NA VAGA 2
VERIFICANDO VAGA 1:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: None - ANTERIOR: None
None ESTÁ NA VAGA 1
VERIFICANDO VAGA 3:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: None - ANTERIOR: None
None ESTÁ NA VAGA 3
VERIFICANDO VAGA 2:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: BEE4R22 - ANTERIOR: BEE4R22
BEE4R22 ESTÁ NA VAGA 2
VERIFICANDO VAGA 1:
NÃO HOUVE 

NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: None - ANTERIOR: None
None ESTÁ NA VAGA 3
VERIFICANDO VAGA 2:
A VAGA ESTÁ OCUPADA AGORA
ATUAL: BEE4R22 - ANTERIOR: None
BEE4R22 ESTÁ NA VAGA 2
VERIFICANDO VAGA 1:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: None - ANTERIOR: None
None ESTÁ NA VAGA 1
VERIFICANDO VAGA 3:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: None - ANTERIOR: None
None ESTÁ NA VAGA 3
VERIFICANDO VAGA 2:
A VAGA ESTÁ LIVRE AGORA
ATUAL: None - ANTERIOR: BEE4R22
None ESTÁ NA VAGA 2
VERIFICANDO VAGA 1:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: None - ANTERIOR: None
None ESTÁ NA VAGA 1
VERIFICANDO VAGA 3:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: None - ANTERIOR: None
None ESTÁ NA VAGA 3
VERIFICANDO VAGA 2:
A VAGA ESTÁ OCUPADA AGORA
ATUAL: BEE4R22 - ANTERIOR: None
BEE4R22 ESTÁ NA VAGA 2
VERIFICANDO VAGA 1:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: None - ANTERIOR: None
None ESTÁ NA VAGA 1
VERIFICANDO VAGA 3:
NÃO HOUVE MUDANÇA NA OCUPAÇÃO DA VAGA
ATUAL: None - ANTERIOR